In [96]:
from wayve.services.data.lakehouse.common.external_access.wayve_delta_table import WayveDeltaTable
import pandas as pd
import numpy as np
from pygeometry import SE3
import pygeometry as pyg
from wayve.core.data.taxonomy import RunId
from pathlib import Path
from bokeh.palettes import Category10
from bokeh.plotting import figure
from bokeh.io import output_file, save
from wayve.core.data.reference_frames import ReferenceFrame
from wayve.core.data.schema.keys.identifiers import RUN_ID_COLUMN
from wayve.robot.slam.geometry.pyg_utils import se3_list_to_df, transform_relative_pose_to_frame
from wayve.services.slam.wayveslam.vo.vo import rename_vo_cols
from wayve.services.slam.wayveslam.run_pose_graph_relax import _unbiased_wheel_odom
from typing import Dict
from scipy.spatial.transform import Rotation as R

In [97]:
def get_estimated_steering_bias(run_id: str) -> float:
    vehicle_calibration_df = WayveDeltaTable.from_database_and_table_name("raw__inference", "targetless_vehicle_calibration").to_pandas_dataframe(run_ids=[run_id])
    return vehicle_calibration_df["vehicle_intrinsics"].loc[0]["steering_bias_deg"]

def get_deployed_steering_bias(run_id: str) -> float:
    vehicle_calibration_df = WayveDeltaTable.from_database_and_table_name("inferred__metadata", "standardised_runtime_vehicle_calibration").to_pandas_dataframe(run_ids=[run_id])
    return vehicle_calibration_df["vehicle_intrinsics"].loc[0]["steering_bias_deg"]

In [98]:
def _get_steering_table_df(run_id: str) -> pd.DataFrame:
    steering_df = WayveDeltaTable.from_database_and_table_name("raw__gen2", "CAN_BUS_GEN2_DATASPEED_MACHE_V1_DBW__SteerReport1").to_pandas_dataframe(run_ids=[run_id])
    return steering_df

def _get_wheel_odometry_table_df(run_id: str) -> pd.DataFrame:
    wo_df = WayveDeltaTable.from_database_and_table_name("inferred__state", "odometry_wheel").to_pandas_dataframe(run_ids=[run_id])
    return wo_df

def get_wheel_odometry_and_steering_df(run_id: str) -> pd.DataFrame:
    # Get the steering angle dataframe
    steering_angle_df = _get_steering_table_df(run_id)

    # Get the wheel odometry spark dataframe
    wo_df = _get_wheel_odometry_table_df(run_id)

    wo_and_steering_df = pd.merge_asof(wo_df, steering_angle_df, on="timestamp_unixus", direction="nearest", tolerance=40_000)

    return wo_and_steering_df

In [99]:
def _unroll_wheel_odom_only(
    pose_df: pd.DataFrame,
    run_id: RunId,
    steering_bias: float,
):
    global_wo = _unbiased_wheel_odom(pose_df, steering_bias, [])
    R_flu_rdf = np.array([[0, 0, 1, 0], [-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 0, 1]], dtype=np.float64)

    # changes from rdf to flu as unbiased wo from pygeometry is in rdf
    G_flu_rdf = SE3(
        dest=ReferenceFrame.GroundNominal.value,
        source=ReferenceFrame.GroundNominal.value,
        matrix=R_flu_rdf,
    )
    relative_wo_flu = transform_relative_pose_to_frame(pyg.global_poses_to_relative(global_wo), G_flu_rdf)
    global_wo_flu = pyg.relative_poses_to_global(relative_wo_flu)
    global_wo_df = se3_list_to_df(global_wo_flu)
    output_df = rename_vo_cols(global_wo_df)
    output_df.loc[:, RUN_ID_COLUMN] = str(run_id)
    return output_df

In [100]:
def plot_trajectories(df: pd.DataFrame, title: str, output_path: Path):
    """Plots a dict of trajectories with no legends"""
    bev_plot = figure(match_aspect=True)
    colors = Category10[10]

    x = df['x_position_m'].values
    y = df['y_position_m'].values

    bev_plot.line(
        x=x,
        y=y,
        line_width=2,
        color=colors[0],
        legend_label="wo",
    )

    bev_plot.title = title

    output_path.parent.mkdir(parents=True, exist_ok=True)
    output_file(output_path)
    save(bev_plot)

def plot_multiple_trajectories(plot_source: Dict[str, pd.DataFrame], title: str) -> figure:
    """Plots a dict of trajectories with no legends"""
    bev_plot = figure(match_aspect=True)
    colors = Category10[10]

    for i, key in enumerate(plot_source):
        x = plot_source[key]['x_position_m'].values
        y = plot_source[key]['y_position_m'].values

        bev_plot.line(
            x=x,
            y=y,
            line_width=2,
            color=colors[i % len(colors)],
            legend_label=key,
        )

    bev_plot.title = title
    return bev_plot

In [101]:
def get_run_ids_for_vehicle_id(start_date: str, end_date: str, vehicle_id: str) -> list:
    est_calib_df = WayveDeltaTable.from_database_and_table_name("raw__inference", "steering_bias_estimate").to_pandas_dataframe()

    filtered_df = est_calib_df[est_calib_df["run_vehicle_id"] == vehicle_id]
    filtered_df = filtered_df.loc[(filtered_df['run_date_iso'].astype('str') >= start_date) & (filtered_df['run_date_iso'].astype('str') <= end_date)]

    return filtered_df["run_id"].tolist()

In [102]:
def build_filename(log_dir: Path, vehicle_id: str, run_id: str, est_sb: float):
    return log_dir / vehicle_id / f"{run_id.replace(vehicle_id, '').replace('/', '')}_unbiased_wo_with_est_sb_{est_sb}_vs_loc.html"

In [103]:
def get_localisation_data_gen2(run_id, start_time_utc_us, end_time_utc_us) -> pd.DataFrame:
    df = (
        WayveDeltaTable.from_database_and_table_name(
            "raw__gen2",
            "robot_localisation_localisation",
        )
        .to_pandas_dataframe(
            run_ids=[run_id],
            columns=[
                "header.timestamp.seconds",
                "header.timestamp.nanos",
                "position.latitude_degs",
                "position.longitude_degs",
            ],
            order_by_run_id_and_timestamp=False,
        )
    )

    # Convert the timestamp to microseconds
    df["timestamp_us"] = (df["seconds"] * 1e6 + df["nanos"] / 1e3).astype("int64")
    df.drop(columns=["seconds", "nanos"], inplace=True)
    df.sort_values("timestamp_us", inplace=True)

    return df[(df["timestamp_us"] >= start_time_utc_us) & (df["timestamp_us"] <= end_time_utc_us)]

def gps_to_ned(df):
    # Constants
    a = 6378137.0  # WGS-84 semimajor axis
    e2 = 0.00669437999014  # WGS-84 first eccentricity squared
    ref_lat_lon_alt = [51.54292044158164, -0.12562772532823005, 0.0]

    # Reference point (latitude, longitude, and altitude)
    lat_ref, lon_ref, alt_ref = np.radians(ref_lat_lon_alt[0]), np.radians(ref_lat_lon_alt[1]), ref_lat_lon_alt[2]

    # Convert degrees to radians for calculations
    df['latitude_rads'] = np.radians(df['latitude_degs'])
    df['longitude_rads'] = np.radians(df['longitude_degs'])

    # Calculate differences
    df['dlat'] = df['latitude_rads'] - lat_ref
    df['dlon'] = df['longitude_rads'] - lon_ref
    df['dalt'] = 0

    # Radius of curvature in the prime vertical
    Rn = a / np.sqrt(1 - e2 * np.sin(lat_ref)**2)

    # Radius of curvature in the meridian
    Rm = Rn * ((1 - e2) / (1 - e2 * np.sin(lat_ref)**2))

    # Convert differences to meters
    df['y_position_m'] = df['dlat'] * (Rm + alt_ref) #north
    df['x_position_m'] = df['dlon'] * (Rn + alt_ref) * np.cos(lat_ref) #east

    # Calculate heading
    df['dx'] = df['x_position_m'].diff()
    df['dy'] = df['y_position_m'].diff()
    df['heading'] = np.arctan2(df['dy'], df['dx'])
    df['heading_deg'] = np.degrees(df['heading'])
    df['heading_deg'] = (df['heading_deg'] + 360) % 360  # Normalize to [0, 360)

    # Select only the NED columns
    ned_df = df[['y_position_m', 'x_position_m', 'heading_deg']]
    ned_df['y_position_m'] = ned_df['y_position_m'] - ned_df['y_position_m'].iloc[0]
    ned_df['x_position_m'] = ned_df['x_position_m'] - ned_df['x_position_m'].iloc[0]
    ned_df['timestamp_unixus'] = df['timestamp_us']

    return ned_df

def get_range_of_steering_biases(x: float):
    return np.arange(-x, x, 5)

In [104]:
def kabsch_calc_rot(points1, points2):
    # Calculate the centroid of the points
    centroid1 = np.mean(points1, axis=0)
    centroid2 = np.mean(points2, axis=0)

    # Subtract the centroids from the points
    points1 -= centroid1
    points2 -= centroid2

    # Calculate the covariance matrix
    H = np.dot(points1.T, points2)

    # Use the SVD to calculate the rotation matrix
    U, S, Vt = np.linalg.svd(H)
    V = Vt.T
    R = np.dot(V, U.T)

    # Ensure a right-handed coordinate system
    if np.linalg.det(R) < 0:
        V[:, -1] *= -1
        R = np.dot(V, U.T)

    return R

def kabsch_align(df1, df2):
    # Drop unnecessary columns
    df1 = df1[["timestamp_unixus", "x_position_m", "y_position_m"]]
    df2 = df2[["timestamp_unixus", "x_position_m", "y_position_m"]]

    df1.rename(columns={"x_position_m": "x_position_m1", "y_position_m": "y_position_m1"}, inplace=True)
    df2.rename(columns={"x_position_m": "x_position_m2", "y_position_m": "y_position_m2"}, inplace=True)

    merged_df = pd.merge_asof(df1, df2, on="timestamp_unixus", direction="nearest", tolerance=20_000)

    # Align the two trajectories using Kabsch algorithm
    df1pos = merged_df[['x_position_m1', 'y_position_m1']].values
    df2pos = merged_df[['x_position_m2', 'y_position_m2']].values

    rot_mat = kabsch_calc_rot(df1pos, df2pos)
    
    df2_aligned = (rot_mat @ df2pos.T).T

    df1 = pd.DataFrame(df1pos, columns=['x_position_m', 'y_position_m'])
    df2_aligned = pd.DataFrame(df2_aligned, columns=['x_position_m', 'y_position_m'])

    return df1, df2_aligned

In [105]:
START_DATE = "2024-07-10"
END_DATE = "2024-07-10"
VEHICLE_ID = "elbe"
LOG_DIR = Path(f"/home/benjin/public-html/wheelodometry/")

In [106]:
run_ids = get_run_ids_for_vehicle_id(START_DATE, END_DATE, VEHICLE_ID)

run_ids = [
    # "orinoco/2024-07-10--14-02-38--gen2-65decb2e-d662-4e99-b843-b34e73265abf",
    # "elbe/2024-07-11--10-50-30--gen2-9efd1ba7-6b72-4ec1-a47b-c38ecf98d37c",
    "elbe/2024-07-11--10-10-30--gen2-f15d171b-2a54-454d-9c58-cac9acdddc71",
    # "elbe/2024-07-10--13-41-09--gen2-c7c9434a-cf0a-4f07-b7fe-cb0063f83b33",
]

SB_OVERWRITE = -3.5

for run_id in run_ids:
    print(run_id)
    # Unbiased WO
    dep_sb = get_deployed_steering_bias(run_id)
    dep_sb = SB_OVERWRITE
    wheel_odometry_df = get_wheel_odometry_and_steering_df(run_id)
    unbiased_wheel_odometry_df = _unroll_wheel_odom_only(wheel_odometry_df, run_id, dep_sb)
    
    # Localisation
    localisation_coords = get_localisation_data_gen2(run_id, 0, float("inf"))
    positions_from_localisation_df = gps_to_ned(localisation_coords)

    # Align trajectories
    try:
        unbiased_wo_df, loc_df = kabsch_align(unbiased_wheel_odometry_df, positions_from_localisation_df)
    except np.linalg.LinAlgError as e:
        print(f"Failed to align trajectories for run_id: {run_id}")
        continue

    log_file = build_filename(LOG_DIR, VEHICLE_ID, run_id, dep_sb)

    trajectories = {
        "Unbiased Wheel Odometry": unbiased_wo_df,
        "Localisation": loc_df
    }

    fig = plot_multiple_trajectories(trajectories, f"Unbiased Wheel Odometry (sb: {dep_sb}) vs Localisation")

    Path(log_file).parent.mkdir(parents=True, exist_ok=True)
    output_file(log_file)
    save(fig)

elbe/2024-07-11--10-10-30--gen2-f15d171b-2a54-454d-9c58-cac9acdddc71


/tmp/ipykernel_2285457/1729164159.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ned_df['y_position_m'] = ned_df['y_position_m'] - ned_df['y_position_m'].iloc[0]
/tmp/ipykernel_2285457/1729164159.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ned_df['x_position_m'] = ned_df['x_position_m'] - ned_df['x_position_m'].iloc[0]
/tmp/ipykernel_2285457/1729164159.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v